In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

The history saving thread hit an unexpected error (OperationalError('database is locked',)).History will not be written to the database.


In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = de5d43e
molecule = {'h2o': 'atmpro', 'co2': 0.0004}
band = [11]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.131813                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.266735
0.000750      2.0  0.009770                  2.0  0.088053
0.001052      3.0  0.006112                  3.0  0.045934
0.001476      4.0  0.001071                  4.0  0.009663
0.002070      5.0 -0.003875                  5.0 -0.016908
0.002904      6.0 -0.008233                  6.0 -0.031935
0.004074      7.0 -0.012189                  7.0 -0.037657
0.005714      8.0 -0.016012                  8.0 -0.039561
0.008015      9.0 -0.019905                  9.0 -0.039610
0.011243     10.0 -0.024035                 10.0 -0.040253
0.015771     11.0 -0.028514                 11.0 -0.041206
0.022122     12.0 -0.033517                 12.0 -0.043686
0.031031     13.0 -0.039586                 13.0 -0.048003
0.043528     14.0 -0.049764                 14.0 -0.057880
0.061057     15.0 -0.061690                 15.0 -0.068903
0.085645     16.0 -0.065718                 16.0 -0.067354
0.120136     17.0 -0.061404                 17.0 -0.052122
0.168516     18.0 -0.056098                 18.0 -0.034287
0.236378     19.0 -0.044777                 19.0 -0.015676
0.331549     20.0 -0.021935                 20.0  0.002416
0.465100     21.0  0.021590                 21.0  0.025737
0.652400     22.0  0.119315                 22.0  0.096726
0.915100     23.0  0.271593                 23.0  0.230195
1.283650     24.0  0.269905                 24.0  0.239880
1.800600     25.0  0.151761                 25.0  0.165195
2.525700     26.0  0.082714                 26.0  0.114476
3.542800     27.0  0.045871                 27.0  0.070838
4.969550     28.0  0.024650                 28.0  0.031653
6.970850     29.0  0.013194                 29.0  0.007861
9.778100     30.0  0.006722                 30.0  0.003272
13.715850    31.0  0.002255                 31.0  0.005187
19.239350    32.0 -0.000621                 32.0  0.003287
26.987250    33.0 -0.001172                 33.0  0.000615
37.855300    34.0 -0.000740                 34.0 -0.000826
53.100050    35.0 -0.000840                 35.0 -0.001338
73.887500    36.0 -0.001263                 36.0 -0.001614
97.662500    37.0 -0.001454                 37.0 -0.001718
121.437500   38.0 -0.001505                 38.0 -0.001722
145.212500   39.0 -0.001962                 39.0 -0.002228
168.987500   40.0 -0.002734                 40.0 -0.003111
192.762500   41.0 -0.003144                 41.0 -0.003557
216.537500   42.0 -0.002945                 42.0 -0.003407
240.312500   43.0 -0.002853                 43.0 -0.003479
264.087500   44.0 -0.002869                 44.0 -0.003698
287.862500   45.0 -0.002938                 45.0 -0.003823
311.637500   46.0 -0.002953                 46.0 -0.003713
335.412500   47.0 -0.002818                 47.0 -0.003541
359.187500   48.0 -0.002524                 48.0 -0.003363
382.962500   49.0 -0.002066                 49.0 -0.003141
406.737500   50.0 -0.001436                 50.0 -0.002807
430.512500   51.0 -0.000659                 51.0 -0.002214
454.287500   52.0  0.000214                 52.0 -0.001338
478.062500   53.0  0.001190                 53.0 -0.000086
501.837500   54.0  0.002291                 54.0  0.001392
525.612500   55.0  0.003543                 55.0  0.003142
549.387500   56.0  0.004983                 56.0  0.005101
573.162500   57.0  0.006667                 57.0  0.007179
596.937500   58.0  0.008878                 58.0  0.009590
620.712500   59.0  0.011150                 59.0  0.011736
644.487500   60.0  0.013407                 60.0  0.013487
668.262500   61.0  0.015687                 61.0  0.014748
692.037500   62.0  0.018012                 62.0  0.015676
715.812500   63.0  0.020436                 63.0  0.016656
739.587500   64.0  0.023029                 64.0  0.018342
76

# Fluxes by Layer

CRD                        CLIRAD                    
                        flug      fldg     fnetg      flug      fldg     fnetg
pressure     level                                                            
0.000000e+00 1     -4.087705  0.000000 -4.087705       NaN       NaN       NaN
1.000000e-08 1           NaN       NaN       NaN -4.022173  0.000002 -4.022171
6.244000e-04 2     -4.087709  0.000014 -4.087695 -4.022179  0.000029 -4.022151
8.759000e-04 3     -4.087711  0.000017 -4.087695 -4.022182  0.000034 -4.022148
1.228600e-03 4     -4.087714  0.000019 -4.087695 -4.022186  0.000040 -4.022146
1.723400e-03 5     -4.087718  0.000023 -4.087695 -4.022192  0.000046 -4.022146
2.417400e-03 6     -4.087722  0.000028 -4.087695 -4.022199  0.000051 -4.022147
3.390900e-03 7     -4.087729  0.000033 -4.087696 -4.022208  0.000057 -4.022151
4.756500e-03 8     -4.087738  0.000040 -4.087698 -4.022220  0.000063 -4.022157
6.672000e-03 9     -4.087750  0.000048 -4.087701 -4.022237  0.000071 -4.022166
9.358900e-03 10    -4.087766  0.000058 -4.087708 -4.022260  0.000082 -4.022179
1.312780e-02 11    -4.087789  0.000070 -4.087719 -4.022292  0.000096 -4.022196
1.841450e-02 12    -4.087821  0.000085 -4.087736 -4.022338  0.000115 -4.022222
2.583020e-02 13    -4.087868  0.000102 -4.087766 -4.022403  0.000142 -4.022261
3.623230e-02 14    -4.087937  0.000123 -4.087815 -4.022498  0.000178 -4.022320
5.082340e-02 15    -4.088048  0.000147 -4.087901 -4.022646  0.000226 -4.022420
7.129060e-02 16    -4.088236  0.000186 -4.088050 -4.022888  0.000301 -4.022587
1.000000e-01 17    -4.088552  0.000278 -4.088274 -4.023269  0.000453 -4.022816
1.402710e-01 18    -4.089052  0.000485 -4.088567 -4.023834  0.000769 -4.023065
1.967600e-01 19    -4.089828  0.000886 -4.088942 -4.024661  0.001367 -4.023294
2.759970e-01 20    -4.091009  0.001646 -4.089362 -4.025857  0.002415 -4.023441
3.871000e-01 21    -4.092733  0.003082 -4.089651 -4.027537  0.004128 -4.023410
5.431000e-01 22    -4.095036  0.005783 -4.089252 -4.029759  0.006826 -4.022934
7.617000e-01 23    -4.096979  0.010817 -4.086162 -4.031621  0.011192 -4.020429
1.068500e+00 24    -4.095148  0.018857 -4.076292 -4.029741  0.017678 -4.012062
1.498800e+00 25    -4.088866  0.026332 -4.062534 -4.023832  0.023999 -3.999834
2.102400e+00 26    -4.081636  0.029953 -4.051683 -4.017004  0.028983 -3.988021
2.949000e+00 27    -4.074688  0.031300 -4.043388 -4.010228  0.033688 -3.976539
4.136600e+00 28    -4.068312  0.031377 -4.036935 -4.003676  0.037103 -3.966573
5.802500e+00 29    -4.062654  0.030584 -4.032071 -3.997536  0.037210 -3.960326
8.139200e+00 30    -4.057630  0.029211 -4.028419 -3.991748  0.033598 -3.958150
1.141700e+01 31    -4.053099  0.027290 -4.025809 -3.986250  0.029371 -3.956879
1.601470e+01 32    -4.049231  0.024650 -4.024581 -3.981378  0.027324 -3.954054
2.246400e+01 33    -4.046531  0.021476 -4.025055 -3.977819  0.026276 -3.951542
3.151050e+01 34    -4.045026  0.018715 -4.026311 -3.975544  0.024661 -3.950883
4.420010e+01 35    -4.044336  0.016913 -4.027423 -3.974087  0.021963 -3.952124
6.200000e+01 36    -4.044823  0.015630 -4.029194 -3.973915  0.018969 -3.954946
8.577500e+01 37    -4.047482  0.014730 -4.032752 -3.976502  0.017011 -3.959492
1.095500e+02 38    -4.051487  0.014641 -4.036846 -3.980930  0.016599 -3.964331
1.333250e+02 39    -4.056301  0.015215 -4.041086 -3.986358  0.017175 -3.969182
1.571000e+02 40    -4.062470  0.015859 -4.046611 -3.993329  0.017870 -3.975459
1.808750e+02 41    -4.071322  0.017010 -4.054312 -4.003452  0.019232 -3.984220
2.046500e+02 42    -4.083426  0.020259 -4.063168 -4.017276  0.023036 -3.994240
2.284250e+02 43    -4.098349  0.026888 -4.071461 -4.034092  0.030256 -4.003836
2.522000e+02 44    -4.116266  0.036769 -4.079497 -4.053944  0.040309 -4.013636
2.759750e+02 45    -4.137862  0.050287 -4.087576 -4.077492  0.053439 -4.024053
2.997500e+02 46    -4.164083  0.068233 -4.095850 -4.105622  0.070802 -4.034820
3.235250e+02 47    -4.194618  0.090452 -4.104166 -4.137965  0.092687 -4.045

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')